In [1]:
import sys
sys.path.append("../")
import pandas as pd
from api.oanda_api import OandaApi
# from infrastructure.instrument_collection import instrumentCollection
from dateutil import parser
# from infrastructure import collect_data
import timeit
# from technicals import pattern_detector
# from technicals import trend_checker
import plotly.graph_objects as go
pd.set_option('display.max_columns', None)

<bound method OandaApi.get_account_summary of <api.oanda_api.OandaApi object at 0x0000017E4A25D2B0>>